In [3]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

  Activating environment at `C:\Users\tge13\Documents\optimal_control_julia\lec_14\Project.toml`


In [4]:
using LinearAlgebra
using ForwardDiff

In [5]:
function hat(v)
    return [0 -v[3] v[2];
            v[3] 0 -v[1];
            -v[2] v[1] 0]
end

hat (generic function with 1 method)

In [6]:
function L(q)
    s = q[1]
    v = q[2:4]
    L = [s    -v';
         v  s*I+hat(v)]
    return L
end

L (generic function with 1 method)

In [7]:
function R(q)
    s = q[1]
    v = q[2:4]
    R = [s    -v';
         v  s*I-hat(v)]
    return R
end

R (generic function with 1 method)

In [8]:
T = Diagonal([1; -ones(3)])
H = [zeros(1,3); I];

In [9]:
function G(q)
    G = L(q)*H
end

function Q(q)
    return H'*(R(q)'*L(q))*H
end

Q (generic function with 1 method)

In [10]:
#Generate a random quaternion
qtrue = randn(4)
qtrue = qtrue/norm(qtrue)

Qtrue = Q(qtrue) #Generate equivalent rotation matrix

3×3 Matrix{Float64}:
 -0.589061   -0.123251    0.798634
 -0.0994094   0.991846    0.0797464
 -0.80195    -0.0324162  -0.59651

In [11]:
#Generate data
vN = randn(3,10) #Generate some random world-frame vectors

#normalize
for k = 1:10
    vN[:,k] .= vN[:,k]./norm(vN[:,k])
end

vB = Qtrue'*vN #generate body-frame vectors

3×10 Matrix{Float64}:
 -0.811372  -0.0285818  -0.935613  …   0.217415  0.673059   0.306185
  0.155662   0.504753    0.326008      0.929509  0.357006  -0.304588
  0.563423   0.862791    0.135451     -0.297898  0.647718  -0.901929

In [12]:
function residual(q)
    r = vN - Q(q)*vB
    return r[:]
end

residual (generic function with 1 method)

In [13]:
#Random initial guess
q = randn(4)
q = q/norm(q)

4-element Vector{Float64}:
  0.2582648374636437
 -0.6482285781773407
  0.09281694601754631
  0.710270370138046

In [20]:
#Gauss-Newton Method
ϕ = ones(3)
iter = 0
while maximum(abs.(ϕ)) > 1e-8
    r = residual(q)
    dr = ForwardDiff.jacobian(residual, q)
    ∇r = dr*G(q)
    @show size(r), size(dr), size(∇r)
    ϕ = -(∇r'*∇r)\(∇r'*r) #3-parameter update computed with gauss-newton
    q = L(q)*[sqrt(1-ϕ'*ϕ); ϕ] #multiplicative update applied to q
    iter += 1
end

(size(r), size(dr), size(∇r)) = ((30,), (30, 4), (30, 3))


In [15]:
iter

7

In [16]:
q-qtrue

4-element Vector{Float64}:
  5.551115123125783e-17
 -2.0816681711721685e-17
 -1.1102230246251565e-16
  5.204170427930421e-18

In [17]:
q+qtrue

4-element Vector{Float64}:
  0.8979276124895488
 -0.12491273472708439
  1.7825312873840287
  0.026552230204964604